In [2]:
import sys
sys.path.append('/Users/Malachite/Documents/UW/ARA/ARA-Plumes/')
from models import PLUME

## Instantiate PLUME object

In [3]:
video_path = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_2/high_2.MP4"
plume_learner = PLUME(video_path=video_path)

plume_leak_source = (1586, 1017)
plume_learner.orig_center = plume_leak_source

seconds = 2

fixed_range = 4*119+34
# img_range = [fixed_range+seconds*plume_learner.fps, plume_learner.tot_frames - plume_learner.fps*13]
img_range = [fixed_range+seconds*plume_learner.fps, 600+fixed_range+seconds*plume_learner.fps]

## Apply Concentric Circles

In [5]:
folder_name = "gauss_time_blur_2"

gauss_space_blur = True
gauss_kernel_size = 251
gauss_space_sigma = 105

gauss_time_blur = True
gauss_time_window = 5
gauss_time_sigma =0.5


mean_smoothing = True
mean_smoothing_sigma = 3

# output = plume_learner.train(img_range=img_range,
#                              fixed_range=fixed_range,
#                              gauss_space_blur=gauss_space_blur,
#                              gauss_kernel_size=gauss_kernel_size,
#                              gauss_space_sigma=gauss_space_sigma,
#                              gauss_time_blur=gauss_time_blur,
#                              gauss_time_window=gauss_time_window,
#                              gauss_time_sigma=gauss_time_sigma,
#                              mean_smoothing=mean_smoothing,
#                              mean_smoothing_sigma=mean_smoothing_sigma)

### Save arrays to pickle file

In [7]:
import pickle

## Save poly coeff

In [ ]:
arrays_dict = {
    'mean': plume_learner.mean_poly,
    'var1': plume_learner.var1_poly,
    'var2': plume_learner.var1_poly
}

pickle_name = folder_name+"/gauss_blur_coeff.pkl"

with open(pickle_name, 'wb') as f:
    pickle.dump(arrays_dict,f)

## Load poly coeff

In [8]:
pickle_path = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_2/"+folder_name+"/gauss_blur_coeff.pkl"

with open(pickle_path, 'rb') as f:
    loaded_arrays = pickle.load(f)

mean_array = loaded_arrays["mean"]
var1_array = loaded_arrays["var1"]
var2_array = loaded_arrays["var2"]

## Apply an Ensemble SINDy

In [9]:
import pysindy as ps
from sklearn.preprocessing import StandardScaler
import numpy as np

In [10]:
start_time = 0
end_time = mean_array.shape[0]/plume_learner.fps
t = np.linspace(start_time,end_time,mean_array.shape[0])

In [11]:
# SINDy model
feature_names = ['a', 'b','c']
ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names,
                 optimizer=ensemble_optimizer)
model.fit(mean_array,t=t,ensemble=True,quiet=True)
model.print()

(a)' = 0.000
(b)' = -24.324 1 + 100.115 b + 334.868 a c + -79.766 b^2
(c)' = -10034.870 1 + 148416587.508 a + 58774.856 b + 42.007 c + -169584133.395 a b + -449387.637 a c + -51621.401 b^2 + -156.968 b c


In [12]:
# SINDy model with normalized coeff
scalar = StandardScaler()
normalized_mean_array = scalar.fit_transform(mean_array)

feature_names = ['a','b','c']
ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names,
                 optimizer=ensemble_optimizer)
model.fit(normalized_mean_array,t=t,ensemble=True,quiet=True)
model.print()

(a)' = -6.097 1 + -190.262 a + -283.794 b + -93.078 c + 544.197 a^2 + 1619.270 a b + 572.035 a c + 1197.221 b^2 + 832.666 b c + 140.115 c^2
(b)' = 5.948 1 + 188.482 a + 279.599 b + 92.058 c + -519.461 a^2 + -1539.611 a b + -539.485 a c + -1132.138 b^2 + -778.353 b c + -128.455 c^2
(c)' = -5.523 1 + -179.371 a + -264.249 b + -87.498 c + 490.889 a^2 + 1445.018 a b + 501.155 a c + 1051.705 b^2 + 710.496 b c + 113.284 c^2
